In [3]:
import os
import sys
import rbm_torch.utils.data_prep as dp

# import rbm_torch.utils.utils
import rbm_torch.analysis.analysis_methods as am

In [2]:
dataset_focus = "pig"
dataset_dir = f"./{dataset_focus}/"

if not os.path.isdir(dataset_dir):
    os.mkdir(dataset_dir)

In [3]:
import data_prep as dp

# Read in raw fasta files. All data is stored in the returned dataframe. Violin out generates a violin plot of sequence lengths for each fasta file.
pig_df = dp.process_raw_fasta_files("b3_cdr3.fasta", "n1_cdr3.fasta", "np1_cdr3.fasta", "np2_cdr3.fasta", "np3_cdr3.fasta", in_dir="/mnt/D1/phage_display_analysis/pig_tissue/fasta files/", out_dir=dataset_dir, violin_out="pid_cdr3_lengths", input_format="fasta")

# names in dataframe correspond to the files that will be generated in prepare_data_files, this just simplifies the names
round_dict = {"np1_cdr3": "np1", "np2_cdr3": "np2", "np3_cdr3": "np3"}
pig_df.replace({"round": round_dict}, inplace=True)

Observed Characters: ['T', 'M', 'N', 'Y', 'V', 'I', 'H', 'A', 'R', 'C', 'Q', 'F', 'G', 'L', 'D', 'P', 'S', '*', 'K', 'X', 'E', 'W']


In [ ]:
pig_datatype = {"focus": "pig", "molecule": "protein", "id": None, "process": "gaps_end", "clusters": 2, "gap_position_indices": [-1, -1], "cluster_indices": [[12, 22], [35, 45]]} # separate into two clusters seqs of len 12-22 and seqs of len 35-45
chars_replace = {"*": "-", "X": "-"}
dp.prepare_data_files(pig_datatype, pig_df, target_dir=dataset_dir, character_conversion=chars_replace, remove_chars=None)  # Creates datafiles in target directory

# generates files b1_c1.fasta, n1_c1.fasta, np1_c1.fasta, np2_c1.fasta, np3_c1.fasta and b1_c2.fasta, n1_c2.fasta, np1_c2.fasta, np2_c2.fasta, np3_c2.fasta

# c1 files are sequences of length 12-22
# c2 files are sequences of lenght 35-45

In [5]:
os.getcwd()

'/home/jonah/PycharmProjects/phage_display_ML/datasets'

In [6]:
# read in data to pandas dataframe and normalize counts within each round as "reads per hundred thousand" = (copy_num/(total reads in dataset/ 100000))
c1_df = am.fetch_data(["np1_c1", "np2_c1", "np3_c1"], dir="./pig/raw_rounds/", threads=6, molecule="protein", normalize_counts=True, normalization_denominator=1e5)

# find sequences carried b/t at least two rounds of negative then positive selection
pig_ct = dp.copynum_topology_faster(c1_df, ["np1_c1", "np2_c1", "np3_c1"], threads_per_task=4)
# write out dataframe in csv format, each sequence is normalized copy number across the 3 rounds
pig_ct.to_csv("./pig/pig_c1_ct.csv", index=False)

Process Time 0.29348158836364746
Process Time 0.10747337341308594
Process Time 0.2554490566253662
Process Time 260.66364550590515


In [7]:
import pandas as pd
import numpy as np
# reads per hundred thousand
pig_pd = pd.read_csv("./pig/pig_c1_ct.csv")

# mean and max copy_number for each sequence
# the np.nanmean and np.nanmax ignore nan values in their calculation
pig_pd["mean"] = pig_pd.apply(lambda row : np.nanmean(np.asarray([row[x] for x in ["np1_c1", "np2_c1", "np3_c1"]])), axis=1)
pig_pd["max"] = pig_pd.apply(lambda row : np.nanmax(np.asarray([row[x] for x in ["np1_c1", "np2_c1", "np3_c1"]])), axis=1)

# Fold calculation between the 3rd and 1st round as well as the 3rd and second round
pig_pd["32_fold"] = pig_pd.apply(lambda row: row["np3_c1"]/row["np2_c1"], axis=1)
pig_pd["31_fold"] = pig_pd.apply(lambda row: row["np3_c1"]/row["np1_c1"], axis=1)

In [8]:
from copy import copy

# selection of enriched seqs
e1 = pig_pd[(pig_pd["32_fold"] > 1.) | (pig_pd["31_fold"] > 2.)]
e2 = e1[((e1["32_fold"] > 1.) | (np.isnan(e1["32_fold"]))) & ((e1["31_fold"] > 2.) | (np.isnan(e1["31_fold"])))]

# must copy dataframe before applying a new function to make a new column
enriched = copy(e2)

# Can only have one number in fasta file, in this case I am using fold. these numbers are further processed below to make weight files
enriched["fold"] = enriched.apply(lambda row: np.nanmax(np.asarray([row["31_fold"]/2., row["32_fold"]])), axis=1)

In [9]:
enriched.head()

,sequence,np1_c1,np2_c1,np3_c1,mean,max,32_fold,31_fold,fold
7,AGLGQKGDARTTDDPIEY----,0.241726,NaN,0.658144,0.449935,0.658144,NaN,2.722688,1.361344
15,ASDHRHGKQEMES---------,0.241726,NaN,0.493608,0.367667,0.493608,NaN,2.042016,1.021008
17,AACKRVKETVDS----------,0.483451,NaN,0.987216,0.735334,0.987216,NaN,2.042016,1.021008
19,AALPFLRAQWYKPMSF------,0.966903,1.948697,15.466377,6.127326,15.466377,7.936778,15.995791,7.997896
25,ARRERGKPANSPINY-------,NaN,1.948697,4.936078,3.442388,4.936078,2.533014,NaN,2.533014


In [11]:
# To avoid overfitting to a specific motif, I further process the sequences by removing sequences within x mutations of any other
from rbm_torch.utils.seq_utils import prune_similar_sequences

# sort to preferentially keep most enriched sequences
enriched.sort_values("fold", ascending=False, inplace=True)

# Returns dataframe with subset of sequences all at least (hamming_threshold) mutations away from any other in the subset
enriched_trimmed = prune_similar_sequences(enriched, hamming_threshold=4, molecule="protein")

Kept 13611 of 14040


In [12]:
dp.dataframe_to_fasta(enriched_trimmed, "./pig/efold.fasta", count_key="fold")

In [13]:
# Get folds to standardize

# from rbm_torch.utils.utils import fasta_read
# seqs, folds, chars, q = fasta_read("./pig/enriched_c1.fasta", "protein", threads=5)

# alternatively

seqs = enriched_trimmed.sequence.tolist()
folds = enriched_trimmed.fold.tolist()


In [14]:
# This is a complicated function, which is my fault, below is a description of how it works
# It takes in as an input the fold, or copy number, (really any set of positive values >= 1 is fine)
# the input is divided into groups by the percentile defined in dividers, you can separate into multiple groups, # of groups = # of dividers + 1
# percentile is calculated from the list of unique values in the input
# target_scaling defines how the sum of each group relates to one another, so a target scaling of [2] for dividers [25] means that values in the input above the divider should add up to 2x the sum of the values below the divider
# there should be an equivalent number of dividers as target_scaling values
# the scale defines how the input values are scaled, the options are log which takes the log of each values and linear which maintains the relative spacing between all values

# Option out_plots is where plots showing the pre and post transfomation of the values are located. File affinity mapping is also generated giving an explicit mapping of input to output values
# The ouptut is the resulting scaled values

# A common error is that the sum of the higher values divided by the sum of the lower values is much higher than the target_scaling value
# A helpful error message will appear advising to change the target_scaling and/or the divider boundaries, example below

std_folds = dp.standardize_affinities(folds, out_plots="./pig/enrich", scale="log", dividers=[25], target_scaling=[3.5])

Warning, values in a higher bracket will be lowered past the boundary. Recommend raising target scaling or changing boundaries
Totals in each bracket: [9341, 4270]
Bracket multipliers: [0.7788925140218654]


NameError: name 'exit' is not defined

In [21]:
# after some experimentation, I arrived at this
std_folds = dp.standardize_affinities(folds, out_plots="./pig/efold", scale="log", dividers=[15], target_scaling=[5.3])

In [22]:
dp.make_weight_file("efold_std", std_folds, dir="./pig/")